# Finetune Comics dataset for emotion classification

## Libraries

In [1]:
# Run this cell only once to install LLaMA-Factory

# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [3]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
# from utils.post_processing import post_process_acc

In [4]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Parameters

In [5]:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [6]:
AM_DIR = os.path.abspath(os.path.join(os.path.join(ROOT_DIR, os.pardir), "am_work"))

In [7]:
DATASET_DIR = os.path.join(ROOT_DIR, "datasets")

In [8]:
LLAMA_FACTORY_DIR = os.path.join(AM_DIR, "coling_2025/LLaMA-Factory")

In [9]:
LLAMA_FACTORY_DIR

'/Utilisateurs/umushtaq/am_work/coling_2025/LLaMA-Factory'

In [10]:
BASE_MODEL = "unsloth/Qwen2-7B-Instruct-bnb-4bit"

In [11]:
TASK = "er"

In [12]:
# TAGS = 1
# TAGS = "wtags" if TAGS == 1 else "wotags"

In [13]:
# CONTEXT = "essay" # essay or paragraph

In [14]:
OUTPUT_DIR = os.path.join(ROOT_DIR, "finetuned_models", f"""comics_{TASK}_{BASE_MODEL.split("/")[1]}""")

In [15]:
OUTPUT_DIR

'/Utilisateurs/umushtaq/er_work/finetuned_models/comics_er_Qwen2-7B-Instruct-bnb-4bit'

## Load Dataset

In [16]:
# *** TRAIN DATASET NAME *** #

train_dataset_name = f"""comics_utterance_train.json"""
test_dataset_name = f"""comics_utterance_test.json"""

#train_dataset_name = f"""PE_{TASK}_{CONTEXT}_train.json"""
train_dataset_file = os.path.join(DATASET_DIR, train_dataset_name)

# *** TEST DATASET NAME *** #

#test_dataset_name = f"""PE_{TASK}_{CONTEXT}_test.json"""
test_dataset_file = os.path.join(DATASET_DIR, test_dataset_name)

In [17]:
train_dataset_file, test_dataset_file

('/Utilisateurs/umushtaq/er_work/datasets/comics_utterance_train.json',
 '/Utilisateurs/umushtaq/er_work/datasets/comics_utterance_test.json')

## Fine-tune Model

In [18]:
if not os.path.exists(os.path.join(ROOT_DIR, "ft_arg_files")):
    os.mkdir(os.path.join(ROOT_DIR, "ft_arg_files"))

In [19]:
# *** TRAIN FILE ***

# model_name = f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}"""

train_file = os.path.join(ROOT_DIR, "ft_arg_files", f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}.json""")

In [20]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [21]:
dataset_info_line

{'file_name': '/Utilisateurs/umushtaq/er_work/datasets/comics_utterance_train.json',
 'columns': {'prompt': 'instruction', 'query': 'input', 'response': 'output'}}

In [22]:
with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["comics_er"] = dataset_info_line

with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

### Training Args

In [23]:
NB_EPOCHS = 3

In [24]:
args = dict(
  stage="sft",                           # do supervised fine-tuning
  do_train=True,
  model_name_or_path=BASE_MODEL,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="comics_er",           # use alpaca and identity datasets
  template="qwen",                     # use llama3 prompt template
  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir=OUTPUT_DIR,                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=3000,                       # save checkpoint every 1000 steps
  learning_rate=5e-5,                    # the learning rate
  num_train_epochs=NB_EPOCHS,            # the epochs of training
  max_samples=5000,                       # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                    # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  fp16=True,                             # use float16 mixed precision training
  report_to="none"                       # discards wandb
)

In [25]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [26]:
train_file

'/Utilisateurs/umushtaq/er_work/ft_arg_files/comics_utterance_Qwen2-7B-Instruct-bnb-4bit.json'

In [27]:
p = subprocess.Popen(["llamafactory-cli", "train", train_file], cwd=LLAMA_FACTORY_DIR)

In [28]:
p.wait()

09/17/2024 20:36:11 - INFO - llamafactory.cli - Initializing distributed tasks at: 127.0.0.1:28716


W0917 20:36:12.734000 139658647946560 torch/distributed/run.py:757] 
W0917 20:36:12.734000 139658647946560 torch/distributed/run.py:757] *****************************************
W0917 20:36:12.734000 139658647946560 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0917 20:36:12.734000 139658647946560 torch/distributed/run.py:757] *****************************************


09/17/2024 20:36:21 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
09/17/2024 20:36:21 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
09/17/2024 20:36:21 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, compute dtype: torch.float16
09/17/2024 20:36:21 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
09/17/2024 20:36:21 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
09/17/2024 20:36:21 - INFO - llamafactory.hparams.parser - Process rank: 1, device: cuda:1, n_gpu: 1, distributed training: True, compute dtype: torch.float16


[INFO|tokenization_utils_base.py:2269] 2024-09-17 20:36:22,157 >> loading file vocab.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/vocab.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 20:36:22,157 >> loading file merges.txt from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/merges.txt
[INFO|tokenization_utils_base.py:2269] 2024-09-17 20:36:22,157 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/tokenizer.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 20:36:22,158 >> loading file added_tokens.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4

09/17/2024 20:36:22 - INFO - llamafactory.data.template - Replace eos token: <|im_end|>
09/17/2024 20:36:22 - INFO - llamafactory.data.template - Replace eos token: <|im_end|>
09/17/2024 20:36:22 - INFO - llamafactory.data.loader - Loading dataset /Utilisateurs/umushtaq/er_work/datasets/comics_utterance_train.json...


...

{'train_runtime': 1738.738, 'train_samples_per_second': 6.049, 'train_steps_per_second': 0.378, 'train_loss': 0.1841895292701605, 'epoch': 3.0}


[INFO|configuration_utils.py:733] 2024-09-17 21:06:24,304 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/config.json
[INFO|configuration_utils.py:800] 2024-09-17 21:06:24,304 >> Model config Qwen2Config {
  "_name_or_path": "unsloth/Qwen2-7B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "pad_token_id": 151643,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
 

***** train metrics *****
  epoch                    =      2.9966
  total_flos               = 101340307GF
  train_loss               =      0.1842
  train_runtime            =  0:28:58.73
  train_samples_per_second =       6.049
  train_steps_per_second   =       0.378


[INFO|modelcard.py:449] 2024-09-17 21:06:25,641 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


0

## Inference on the fine-tuned model

In [29]:
OUTPUT_DIR

'/Utilisateurs/umushtaq/er_work/finetuned_models/comics_er_Qwen2-7B-Instruct-bnb-4bit'

In [30]:
os.listdir(OUTPUT_DIR)

['trainer_log.jsonl',
 'checkpoint-657',
 'README.md',
 'adapter_model.safetensors',
 'adapter_config.json',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'added_tokens.json',
 'vocab.json',
 'merges.txt',
 'tokenizer.json',
 'training_args.bin',
 'train_results.json',
 'all_results.json',
 'trainer_state.json']

In [31]:
args = dict(
  model_name_or_path=BASE_MODEL, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=OUTPUT_DIR,            # load the saved LoRA adapters
  template="qwen",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)


In [32]:
model = ChatModel(args)

[INFO|tokenization_utils_base.py:2269] 2024-09-17 21:06:28,952 >> loading file vocab.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/vocab.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 21:06:28,954 >> loading file merges.txt from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/merges.txt
[INFO|tokenization_utils_base.py:2269] 2024-09-17 21:06:28,956 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/tokenizer.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 21:06:28,958 >> loading file added_tokens.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4

09/17/2024 21:06:29 - INFO - llamafactory.data.template - Replace eos token: <|im_end|>


[INFO|configuration_utils.py:733] 2024-09-17 21:06:29,256 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/config.json
[INFO|configuration_utils.py:800] 2024-09-17 21:06:29,260 >> Model config Qwen2Config {
  "_name_or_path": "unsloth/Qwen2-7B-Instruct-bnb-4bit",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "pad_token_id": 151643,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "

09/17/2024 21:06:29 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.
09/17/2024 21:06:29 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
09/17/2024 21:06:29 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:398] 2024-09-17 21:06:29,523 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3678] 2024-09-17 21:06:29,527 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2-7B-Instruct-bnb-4bit/snapshots/26ac6deb2f1f4929ecc7ece49178be31aa1e0755/model.safetensors
[INFO|modeling_utils.py:1606] 2024-09-17 21:06:29,560 >> Instantiating Qwen2ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1038] 2024-09-17 21:06:29,564 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "pad_token_id": 151643
}

[INFO|quantizer_bnb_4bit.py:106] 2024-09-17 21:06:30,712 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization
[INFO|modeling_utils.py:4507] 2024-09-17 21:06

09/17/2024 21:06:33 - INFO - llamafactory.model.model_utils.attention - Using torch SDPA for faster training and inference.
09/17/2024 21:06:33 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/er_work/finetuned_models/comics_er_Qwen2-7B-Instruct-bnb-4bit
09/17/2024 21:06:33 - INFO - llamafactory.model.loader - all params: 7,635,801,600


In [33]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

In [34]:
test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append("\nUser:" + sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [ ]:
test_predictions = []

for prompt in tqdm(test_prompts):

    messages = []
    messages.append({"role": "user", "content": prompt})

    response = ""
    
    for new_text in model.stream_chat(messages):
        #print(new_text, end="", flush=True)
        response += new_text
        #print()
    test_predictions.append({"role": "assistant", "content": response})

    torch_gc()

  0%|          | 0/1776 [00:00<?, ?it/s]

In [ ]:
with open(os.path.join(OUTPUT_DIR, f"""comics_{TASK}_results_{NB_EPOCHS}.pickle"""), 'wb') as fh:
    results_d = {"ground_truths": test_grounds,
                 "predictions": test_predictions    
        
    }
    pickle.dump(results_d, fh)

In [ ]:
os.listdir(OUTPUT_DIR)

## Post-processing

In [ ]:
NB_EPOCHS

In [ ]:
with open(os.path.join(OUTPUT_DIR, f"""comics_{TASK}_results_{NB_EPOCHS}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [ ]:
grounds = results["ground_truths"]
preds = results["predictions"]

In [ ]:
len(grounds), len(preds)

In [ ]:
grounds_l = []

for i in range(len(grounds)):
    grounds_l.append(json.loads(grounds[i])['list_emotion_classes'])

In [ ]:
len(grounds_l)

In [ ]:
grounds_l

In [ ]:
preds = [x["content"] for x in preds]   

In [ ]:
preds_l = []

for i in range(len(preds)):
    preds_l.append(json.loads(preds[i])['list_emotion_classes'])

In [ ]:
len(preds_l)

In [ ]:
for idx, (i,j) in enumerate(zip(grounds_l, preds_l)):
    if len(i) != len(j):
        print(idx)

In [ ]:
# def opposite_acc(component_type):

#     if component_type == "Premise":
#         return "Claim"
#     elif component_type == "Claim":
#         return "Premise"
#     elif component_type == "MajorClaim":
#         return "Claim"

# def harmonize_preds_acc(grounds, preds):

#     l1, l2 = len(preds), len(grounds)
#     if l1 < l2:
#         diff = l2 - l1
#         preds = preds + [opposite_acc(x) for x in grounds[l1:]]
#     else:
#         preds = preds[:l2]
        
#     return preds 

# def post_process_acc(results):

#     grounds = results["ground_truths"]
#     preds = results["predictions"]
    
#     grounds = [json.loads(x)["component_types"] for x in grounds]  
    
#     preds = [x["content"] for x in preds]    
#     preds = [json.loads(x)["component_types"] for x in preds]
    
#     for i,(x,y) in enumerate(zip(grounds, preds)):
    
#         if len(x) != len(y):
            
#             preds[i] = harmonize_preds_acc(x, y)
            
#     task_preds = [item for row in preds for item in row]
#     task_grounds = [item for row in grounds for item in row]

#     return task_grounds, task_preds

In [ ]:
# "Anger" (AN), "Disgust" (DI), "Fear" (FE), "Sadness" (SA), "Surprise" (SU) or "Joy" (JO)

In [ ]:
def opposite_acc(component_type):

    if component_type == "AN":
        return "SU"
    elif component_type == "DI":
        return "JO"
    elif component_type == "FE":
        return "SA"
    elif component_type == "SA":
        return "AN"
    elif component_type == "SU":
        return "DI"
    elif component_type == "JO":
        return "FE"
    elif component_type == "Neutral":
        return "SA"

def harmonize_preds_acc(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite_acc(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [ ]:
for i,(x,y) in enumerate(zip(grounds_l, preds_l)):

    if len(x) != len(y):
        
        preds_l[i] = harmonize_preds_acc(x, y)

In [ ]:
task_preds = [item for row in preds_l for item in row]
task_grounds = [item for row in grounds_l for item in row]

In [ ]:
task_preds

In [ ]:
task_grounds

In [ ]:
task_grounds = ['Neutral' if x == ['Neutral'] else x for x in task_grounds]
task_preds = ['Neutral' if x == ['Neutral'] else x for x in task_preds]

In [ ]:
# sanity check: 
len(task_preds) == len(task_grounds)

In [ ]:
len(task_preds)

In [ ]:
len(task_grounds)

In [ ]:
#task_grounds, task_preds = post_process_acc(results)

In [ ]:
# sanity check: 
#len(task_preds) == len(task_grounds)

## Results

In [ ]:
print(classification_report(task_grounds, task_preds, digits=3))

In [ ]:
with open(f"""{OUTPUT_DIR}/classification_report_llama3.1_5.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(task_grounds, task_preds, output_dict=True), fh)